# Overview
In this section, you will create a managed dataset in Vertex AI, train an AutoML model, evaluate the model, and examine the parameters in the logs to understand how to recreate the model if needed.

## Create Vertex AI Managed Dataset
In order to use Vertex AI AutoML, you must first create a managed dataset. Note - we are using the UI for these exercises, but all of these tasks can also be accomplished using the Vertex AI Python SDK. 

1. In Vertex AI console, navigate to Datasets
2. Select "Create" and give your dataset a name, select Data Type as <b>Tabular</b>, <b>Regression / Classification</b> as the Objective. When choosing your region, choose the same region that you created your Vertex AI workbench managed notebook in, to ensure consistencies in regions. Click "Create"
![](./managed_dataset.png)
3. Now, let's add data to the this dataset. Choose "Select a table or view from BigQuery". Type in the path to the previously created training table in BigQuery: the format will be {PROJECT_ID}.{DATASET}.{TABLE}, then select create. You should see the information about this dataset now available:
![](./dataset_created.png)
You can now also click on "Generate Statistics" to generate a data profile around the data.

Alternatively, you can use the Vertex AI SDK to create the dataset

Link to documentation: https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/create-dataset#aiplatform_create_dataset_tabular_bigquery_sample-python

See example below:

In [1]:
def create_and_import_dataset_tabular_bigquery_sample(
    display_name: str,
    project: str,
    location: str,
    bq_source: str,
):

    aiplatform.init(project=project, location=location)

    dataset = aiplatform.TabularDataset.create(
        display_name=display_name,
        bq_source=bq_source,
    )

    dataset.wait()

    print(f'\tDataset: "{dataset.display_name}"')
    print(f'\tname: "{dataset.resource_name}"')

## Model Training with AutoML
1. Click on "Train New Model"
2. Select Classification as the objective, choose AutoML as the training method
4. Select train a new model, give the model a unique name, select label as the Target Column. Optionally, you can export a test dataset to BigQuery (we also have a separate test dataset reserved, so no need to do this). If you expand advanced options, you can also choose to change some of the default configurations. 
5. For Training Options, proceed with the default settings (unless you want to change them)
6. Provide a budget of 3 node hours, and enable early stopping
7. Click Start Training
8. Navigate to Training on the Vertex AI sidebar - you'll see your job has successfully started, and you can drill into it to get specific details and status. 
![](./automl_train.png)

Alternatively, you can use the Vertex AI SDK to train the model

Link to documentation: https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/train-model

See example below:

In [2]:
def create_training_pipeline_tabular_classification_sample(
    project: str,
    display_name: str,
    dataset_id: str,
    location: str = "us-central1",
    model_display_name: str = None,
    target_column: str = "target_column",
    training_fraction_split: float = 0.8,
    validation_fraction_split: float = 0.1,
    test_fraction_split: float = 0.1,
    budget_milli_node_hours: int = 8000,
    disable_early_stopping: bool = False,
    sync: bool = True,
):
    aiplatform.init(project=project, location=location)

    tabular_classification_job = aiplatform.AutoMLTabularTrainingJob(
        display_name=display_name, optimization_prediction_type="classification"
    )

    my_tabular_dataset = aiplatform.TabularDataset(dataset_name=dataset_id)

    model = tabular_classification_job.run(
        dataset=my_tabular_dataset,
        target_column=target_column,
        training_fraction_split=training_fraction_split,
        validation_fraction_split=validation_fraction_split,
        test_fraction_split=test_fraction_split,
        budget_milli_node_hours=budget_milli_node_hours,
        model_display_name=model_display_name,
        disable_early_stopping=disable_early_stopping,
        sync=sync,
    )

    model.wait()

    print(model.display_name)
    print(model.resource_name)
    print(model.uri)
    return model


Nice! Let this job run - in the meantime, head over to the BQML notebook and continue with the other training exercises whilst this job runs. 

## Evaluate model
Once the model has finished training, click on it to explore the evaluation results. 

You can see the evaluation metrics, as well as the feature attributions - which features were most important to the model prediction results. 

![](./automl_eval.png)

If you navigate to the Vertex AI model registry, you will also see that the model is now available in the model registry. 

Alternatively, you can use the Vertex AI SDK to evaluate the model

Link to documentation: https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/evaluate-model

See example below:

In [ ]:
from google.cloud import aiplatform


def get_model_evaluation_tabular_classification_sample(
    project: str,
    model_id: str,
    evaluation_id: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    """
    To obtain evaluation_id run the following commands where LOCATION
    is the region where the model is stored, PROJECT is the project ID,
    and MODEL_ID is the ID of your model.

    model_client = aiplatform.gapic.ModelServiceClient(
        client_options={
            'api_endpoint':'LOCATION-aiplatform.googleapis.com'
            }
        )
    evaluations = model_client.list_model_evaluations(parent='projects/PROJECT/locations/LOCATION/models/MODEL_ID')
    print("evaluations:", evaluations)
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.ModelServiceClient(client_options=client_options)
    name = client.model_evaluation_path(
        project=project, location=location, model=model_id, evaluation=evaluation_id
    )
    response = client.get_model_evaluation(name=name)
    print("response:", response)


## AutoML Parameters
Source: https://cloud.google.com/automl-tables/docs/logging

Once the training job is finished, you can navigate to get the parameters from the logs.

* Navigate to the training job and click on it
* Select Version Details
* Next to Model hyperparameters, click on Model
* This will launch into Cloud Logging - click Run Query
* Copy the results to clipboard and paste into a new screen


We can see it was a boosted tree model, and we can use these hyperparameters to recreate our own boosted tree if we didn't want to use the AutoML model 

In [ ]:
# Example:

{
  "insertId": "jpknobf3gy1i0",
  "jsonPayload": {
    "@type": "type.googleapis.com/google.cloud.automl.master.TablesModelStructure",
    "modelParameters": [
      {
        "hyperparameters": {
          "model_type": "boosted_trees",
          "max_tree_depth": 3,
          "l2": 0,
          "tree_complexity": 0.10000000149011612,
          "num_trees": 50,
          "l1": 3
        }
      },
      {
        "hyperparameters": {
          "tree_complexity": 0.10000000149011612,
          "l1": 3,
          "num_trees": 50,
          "model_type": "boosted_trees",
          "l2": 0,
          "max_tree_depth": 3
        }
      },
      {
        "hyperparameters": {
          "l1": 3,
          "max_tree_depth": 3,
          "l2": 0,
          "tree_complexity": 0.10000000149011612,
          "model_type": "boosted_trees",
          "num_trees": 50
        }
      },
      {
        "hyperparameters": {
          "model_type": "boosted_trees",
          "max_tree_depth": 3,
          "l1": 3,
          "tree_complexity": 0.10000000149011612,
          "l2": 0,
          "num_trees": 50
        }
      },
      {
        "hyperparameters": {
          "model_type": "boosted_trees",
          "l2": 0,
          "max_tree_depth": 3,
          "l1": 3,
          "num_trees": 50,
          "tree_complexity": 0.10000000149011612
        }
      },
      {
        "hyperparameters": {
          "l1": 3,
          "l2": 0.30000001192092896,
          "model_type": "boosted_trees",
          "max_tree_depth": 3,
          "tree_complexity": 0.30000001192092896,
          "num_trees": 50
        }
      },
      {
        "hyperparameters": {
          "max_tree_depth": 3,
          "l2": 0.30000001192092896,
          "tree_complexity": 0.30000001192092896,
          "num_trees": 50,
          "l1": 3,
          "model_type": "boosted_trees"
        }
      },
      {
        "hyperparameters": {
          "max_tree_depth": 3,
          "l1": 3,
          "tree_complexity": 0.30000001192092896,
          "num_trees": 50,
          "model_type": "boosted_trees",
          "l2": 0.30000001192092896
        }
      },
      {
        "hyperparameters": {
          "l2": 0.30000001192092896,
          "max_tree_depth": 3,
          "model_type": "boosted_trees",
          "tree_complexity": 0.30000001192092896,
          "num_trees": 50,
          "l1": 3
        }
      },
      {
        "hyperparameters": {
          "max_tree_depth": 3,
          "num_trees": 50,
          "tree_complexity": 0.30000001192092896,
          "l2": 0.30000001192092896,
          "l1": 3,
          "model_type": "boosted_trees"
        }
      },
      {
        "hyperparameters": {
          "model_type": "boosted_trees",
          "l2": 0,
          "l1": 0.30000001192092896,
          "tree_complexity": 1,
          "num_trees": 50,
          "max_tree_depth": 3
        }
      },
      {
        "hyperparameters": {
          "l1": 0.30000001192092896,
          "tree_complexity": 1,
          "model_type": "boosted_trees",
          "num_trees": 50,
          "l2": 0,
          "max_tree_depth": 3
        }
      },
      {
        "hyperparameters": {
          "l2": 0,
          "l1": 0.30000001192092896,
          "max_tree_depth": 3,
          "tree_complexity": 1,
          "num_trees": 50,
          "model_type": "boosted_trees"
        }
      },
      {
        "hyperparameters": {
          "l2": 0,
          "tree_complexity": 1,
          "model_type": "boosted_trees",
          "max_tree_depth": 3,
          "l1": 0.30000001192092896,
          "num_trees": 50
        }
      },
      {
        "hyperparameters": {
          "max_tree_depth": 3,
          "l2": 0,
          "tree_complexity": 1,
          "model_type": "boosted_trees",
          "l1": 0.30000001192092896,
          "num_trees": 50
        }
      },
      {
        "hyperparameters": {
          "tree_complexity": 0.30000001192092896,
          "max_tree_depth": 3,
          "l2": 0.30000001192092896,
          "model_type": "boosted_trees",
          "num_trees": 100,
          "l1": 0.30000001192092896
        }
      },
      {
        "hyperparameters": {
          "tree_complexity": 0.30000001192092896,
          "model_type": "boosted_trees",
          "num_trees": 100,
          "l1": 0.30000001192092896,
          "max_tree_depth": 3,
          "l2": 0.30000001192092896
        }
      },
      {
        "hyperparameters": {
          "max_tree_depth": 3,
          "l2": 0.30000001192092896,
          "num_trees": 100,
          "l1": 0.30000001192092896,
          "model_type": "boosted_trees",
          "tree_complexity": 0.30000001192092896
        }
      },
      {
        "hyperparameters": {
          "model_type": "boosted_trees",
          "max_tree_depth": 3,
          "l1": 0.30000001192092896,
          "num_trees": 100,
          "l2": 0.30000001192092896,
          "tree_complexity": 0.30000001192092896
        }
      },
      {
        "hyperparameters": {
          "l2": 0.30000001192092896,
          "l1": 0.30000001192092896,
          "num_trees": 100,
          "tree_complexity": 0.30000001192092896,
          "model_type": "boosted_trees",
          "max_tree_depth": 3
        }
      },
      {
        "hyperparameters": {
          "num_trees": 250,
          "l1": 1,
          "model_type": "boosted_trees",
          "tree_complexity": 0.10000000149011612,
          "l2": 0.10000000149011612,
          "max_tree_depth": 3
        }
      },
      {
        "hyperparameters": {
          "l1": 1,
          "l2": 0.10000000149011612,
          "tree_complexity": 0.10000000149011612,
          "num_trees": 250,
          "model_type": "boosted_trees",
          "max_tree_depth": 3
        }
      },
      {
        "hyperparameters": {
          "l2": 0.10000000149011612,
          "tree_complexity": 0.10000000149011612,
          "l1": 1,
          "max_tree_depth": 3,
          "num_trees": 250,
          "model_type": "boosted_trees"
        }
      },
      {
        "hyperparameters": {
          "l2": 0.10000000149011612,
          "num_trees": 250,
          "tree_complexity": 0.10000000149011612,
          "model_type": "boosted_trees",
          "l1": 1,
          "max_tree_depth": 3
        }
      },
      {
        "hyperparameters": {
          "tree_complexity": 0.10000000149011612,
          "l1": 1,
          "num_trees": 250,
          "l2": 0.10000000149011612,
          "max_tree_depth": 3,
          "model_type": "boosted_trees"
        }
      }
    ]
  },
  "resource": {
    "type": "cloudml_job",
    "labels": {
      "project_id": "leedeb-experimentation",
      "job_id": "1734134346165518336",
      "region": "us-central1"
    }
  },
  "timestamp": "2023-01-18T15:26:57.209204Z",
  "severity": "INFO",
  "labels": {
    "log_type": "automl_tables"
  },
  "logName": "projects/leedeb-experimentation/logs/automl.googleapis.com%2Fmodel",
  "receiveTimestamp": "2023-01-18T18:34:08.777423339Z"
}